# Converting Yelp Dataset json files to CSV

Medium Article - [Convert Yelp Dataset to CSV using Pandas](https://link.medium.com/0k0DEb3Qy1)

[GitHub Repo](https://github.com/gyhou/yelp_dataset)

## Data Source
https://www.yelp.com/dataset/
- Unzip the first .tar file, it will extract a single file
- Add .tar to the end of the file's name to unzip it again
- The second file will extract 6 json files

In [3]:
# Need to upgrade pandas to version 0.25 to use "explode" function
# !pip install --upgrade pandas

In [1]:
import pandas as pd
pd.__version__

'1.4.1'

## To match the review texts with business info, we need to merge the two json files

In [3]:
# Set up your local path
business_json_path = 'business.json'
business = pd.read_json(business_json_path, lines=True)
print(business.shape)
business.head()

(192609, 14)


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,1SWheh84yJXfytovILXOAQ,Arizona Biltmore Golf Club,2818 E Camino Acequia Drive,Phoenix,AZ,85016,33.522143,-112.018481,3.0,5,0,{'GoodForKids': 'False'},"Golf, Active Life",None
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289,2.5,128,1,"{'RestaurantsReservations': 'True', 'GoodForMe...","Specialty Food, Restaurants, Dim Sum, Imported...","{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W..."
2,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,"10110 Johnston Rd, Ste 15",Charlotte,NC,28210,35.092564,-80.859132,4.0,170,1,"{'GoodForKids': 'True', 'NoiseLevel': 'u'avera...","Sushi Bars, Restaurants, Japanese","{'Monday': '17:30-21:30', 'Wednesday': '17:30-..."
3,xvX2CttrVhyG2z1dFg_0xw,Farmers Insurance - Paul Lorenz,"15655 W Roosevelt St, Ste 237",Goodyear,AZ,85338,33.455613,-112.395596,5.0,3,1,None,"Insurance, Financial Services","{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ..."
4,HhyxOkGAM07SRYtlQ4wMFQ,Queen City Plumbing,"4209 Stuart Andrew Blvd, Ste F",Charlotte,NC,28217,35.190012,-80.887223,4.0,4,1,"{'BusinessAcceptsBitcoin': 'False', 'ByAppoint...","Plumbing, Shopping, Local Services, Home Servi...","{'Monday': '7:0-23:0', 'Tuesday': '7:0-23:0', ..."


In [5]:
# There are significant amount of businesses that are not open anymore
business.is_open.value_counts()

1    158525
0     34084
Name: is_open, dtype: int64

In [4]:
# Keep only business that are still open
# Drop columns that may not be relavent
df_business = business[business['is_open']==1].drop(['hours','is_open','review_count'], axis=1)
print(df_business.shape)
df_business.head()

(158525, 11)


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,attributes,categories
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289,2.5,"{'RestaurantsReservations': 'True', 'GoodForMe...","Specialty Food, Restaurants, Dim Sum, Imported..."
2,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,"10110 Johnston Rd, Ste 15",Charlotte,NC,28210,35.092564,-80.859132,4.0,"{'GoodForKids': 'True', 'NoiseLevel': 'u'avera...","Sushi Bars, Restaurants, Japanese"
3,xvX2CttrVhyG2z1dFg_0xw,Farmers Insurance - Paul Lorenz,"15655 W Roosevelt St, Ste 237",Goodyear,AZ,85338,33.455613,-112.395596,5.0,None,"Insurance, Financial Services"
4,HhyxOkGAM07SRYtlQ4wMFQ,Queen City Plumbing,"4209 Stuart Andrew Blvd, Ste F",Charlotte,NC,28217,35.190012,-80.887223,4.0,"{'BusinessAcceptsBitcoin': 'False', 'ByAppoint...","Plumbing, Shopping, Local Services, Home Servi..."
5,68dUKd8_8liJ7in4aWOSEA,The UPS Store,"Credit Valley Town Plaza, F2 - 6045 Creditview Rd",Mississauga,ON,L5V 0B1,43.599475,-79.711584,2.5,"{'BusinessParking': '{'garage': False, 'street...","Shipping Centers, Couriers & Delivery Services..."


### Find relevant categories

One way to split the reviews is by category

In [7]:
# df.explode requires pandas ver 0.25
# Create one row for each series that contain comma-separated items
df_explode = df_business.assign(categories = df_business.categories.str.split(', ')).explode('categories')
df_explode.sample(3)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,attributes,categories
161086,6u0rUgAS7PfjFTx5jHFGog,Panera Bread,14035 E Independence Expy,Indian Trail,NC,28079,35.078224,-80.653957,2.5,"{'RestaurantsGoodForGroups': 'True', 'GoodForK...",Soup
26653,35JoA_jX8eoenORiOVwe9Q,Southwestern Eye Center,"5251 W Campbell Ave, Ste 102",Phoenix,AZ,85031,33.501887,-112.172220,5.0,"{'BusinessParking': '{'garage': False, 'street...",Shopping
86822,CW2RlvudJ1Q18_dzett5KA,Atomic Air,3570 W Dewey Dr,Las Vegas,NV,89118,36.231208,-115.215509,5.0,{'BusinessAcceptsCreditCards': 'True'},Heating & Air Conditioning/HVAC


In [11]:
print('Total number of categories:', len(df_explode.categories.value_counts()))
print('Top 10 categories:')
df_explode.categories.value_counts()[:10]

Total number of categories: 1290
Top 10 categories:


Restaurants                  42237
Shopping                     26734
Food                         23208
Home Services                18455
Beauty & Spas                16545
Health & Medical             15875
Local Services               12830
Automotive                   11955
Nightlife                     9396
Event Planning & Services     8960
Name: categories, dtype: int64

In [13]:
# Finding categories that contains RV
df_explode[df_explode['categories'].str.contains('RV', case=True, na=False)].categories.value_counts()

RV Repair     146
RV Dealers    123
RV Rental      81
RV Parks       60
Name: categories, dtype: int64

In [7]:
# Keep only business with categories that are RV related (including Campgrounds)
business_RV = df_business[df_business['categories'].str.contains(
                         'RV Repair|RV Dealers|RV Rental|RV Parks|Campgrounds', case=False, na=False)]
print(business_RV.shape)
business_RV.head()

(320, 11)


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,attributes,categories
1143,q0B39iv1bs16PO--eiMfIw,Hobo Camper Country,4020 Edmonton Trail NE,Calgary,AB,T2E 3P6,51.088523,-114.051507,1.5,None,"RV Repair, Auto Repair, Automotive"
1458,SW_bePWPlMZJZGQ4eT9vrA,E's RV Appliance Repair,"1959 S Power Rd, Ste 103-238",Mesa,AZ,85206,33.379981,-111.687220,5.0,"{'ByAppointmentOnly': 'True', 'BusinessAccepts...","Automotive, Water Heater Installation/Repair, ..."
2292,4mIyRDV4a1krLYC7Nfn_DQ,Montréal Sud KOA,130 Montée Monette,Saint-Philippe,QC,J0L 2K0,45.346753,-73.489582,3.5,None,"Campgrounds, Hotels & Travel, RV Parks, Event ..."
3180,AtIyt5nYQ_eLPdOnLukmXQ,Mini Self Storage Stephanie,5280 Stephanie St,Las Vegas,NV,89122,36.093833,-115.045345,4.0,{'BusinessAcceptsCreditCards': 'True'},"Local Services, RV Parks, Self Storage, Hotels..."
3482,4Yj5O3Jl3w9tcw844fEzoQ,Latham RV Phoenix,1039 E Broadway Rd,Mesa,AZ,85204,33.399033,-111.797238,2.5,None,"Hotels & Travel, RV Dealers, Active Life, RV R..."


### Loading Massive file as chunks in Pandas

With a huge file like yelp dataset, loading all the data at once will most likely crash your computer.

Luckily, pandas has an option to load large data by segmenting the file into smaller chunks.

In [2]:
# Set up your local path
review_json_path = '../data/yelp_dataset/yelp_academic_dataset_review.json'

In [3]:
import pandas as pd

# Set chunk size (smaller if dataset is smaller)
# 2019 Yelp review.json has more than 6 million reviews(rows)
size = 100000
review = pd.read_json(review_json_path, lines=True,
                      # identifying the data type of each column can reduce memory usage
                      dtype={'review_id':str,'user_id':str,'business_id':str,'stars':int,
                             'date':str,'text':str,'useful':int,'funny':int,'cool':int},
                      chunksize=size)

### Merge Review.json and Business.json file

By merging only the relevant businesses to the review file, the final dataset will only consist of reviews from those businesses

In [6]:
from tqdm import tqdm

In [7]:
# There are multiple chunks to be read
chunk_list = []
for chunk in tqdm(review):
    print(chunk)
    # Drop columns that aren't needed
    chunk = chunk.drop(['review_id','useful','funny','cool'], axis=1)
    # Renaming column name to avoid conflict with business overall star rating
    chunk = chunk.rename(columns={'stars': 'review_stars'})
    # Inner merge with edited business file so only reviews related to the business remain
   # chunk_merged = pd.merge(business_RV, chunk, on='business_id', how='inner')
    # Show feedback on progress
    print(f"{chunk_merged.shape[0]} out of {size:,} related reviews")
    chunk_list.append(chunk_merged)
    
# After trimming down the review file, concatenate all relevant data back to one dataframe
df = pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)
print(df.shape)
df.sample(3)

0it [00:00, ?it/s]

                    review_id                 user_id             business_id  \
0      KU_O5udG6zpxOg-VcAEodg  mh_-eMZ6K5RLWhZyISBhwA  XQfwVwDr-v0ZS3_CbbE5Xw   
1      BiTunyQ73aT9WBnpR9DZGw  OyoGAe7OKpv6SyGZT5g77Q  7ATYjTIgM3jUlt4UM3IypQ   
2      saUsX_uimxRlCVr67Z4Jig  8g_iMtfSiwikVnbP2etR0A  YjUWPpI6HXG530lwP-fb2A   
3      AqPFMleE6RsU23_auESxiA  _7bHUi9Uuf5__HHc_Q8guQ  kxX2SOes4o-D3ZQBkiMRfA   
4      Sx8TMOWLNuJBWer-0pcmoA  bcjbaE6dDog4jkNY91ncLQ  e4Vwtrqf-wpJfwesgvdgxQ   
...                       ...                     ...                     ...   
99995  pAEbIxvr6ebx2bHc1XvguA  SMH5CeiLvKx61lKwtLZ_PA  lV0k3BnslFRkuWD_kbKd0Q   
99996  xH1AoE-4nf2ECGQJRjO4_g  2clTdtp-BjphxLjN83CpUA  G0xz3kyRhRi6oZl7KfR0pA   
99997  GatIbXTz-WDru5emONUSIg  MRrN6DH3QGCFcDv5RENYVg  C4lZdhasjZVQyDlOiXY1sA   
99998  6NfkodAdhvI89xONXuBC3A  rnNQzeKJbvqVCsYsL10mkQ  dChRGpit9fM_kZK5pafNyA   
99999  sJ1BMq7lkKgOWEFx3n6ZRw  _BcWyKQL16ndpBdggh2kNA  hMcgO98QaOFmQVTfCUeGzw   

       stars  useful  funny

NameError: name 'chunk_merged' is not defined

### Convert the new dataframe into a CSV file

By doing so, the smaller dataset can be loaded and shared much more readily

In [10]:
df.to_csv("yelp_reviews_RV_categories.csv", index=False)